Testing delta probability for weeklies and monthlies\n",
    - Does delta match probability of occurrence\n",
    - Check Kelly criterion for long straddles/delta neutral back ratios\n",
        - To calculate expected losses and gains, use options pricing module to interpolate prices at given percentage\n",
        moves and use the normal pdf as the probability weightings\n
Check earnings returns post announcement factors\n",
    - Factors:\n",
        - Number of times beaten earnings (Dummy Variable)\n",
        - Consecutive earnings beats\n",
        - Consecutive earnings upsets\n",
        - 3 Month Trend before earnings\n",
        - YTD Trend\n",
        - Momentum of monthly returns (20 day, 60 day)\n"
        
        
price-to-book (P / B), price-to-earnings (P / E), price-to-free cashflow (P / FCF) and enterprise value-to-EBITDA (EV / EBITDA

In [1]:
import datetime as dt

import numpy as np
import pandas as pd
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *
# ts = TimeSeries(key='5HZEUI5AFJB06BUK',output_format='pandas')
import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *
import plotly
import os
import pandas_market_calendars as mcal
import json
from pandas.io.json import json_normalize
import urllib.request as req
import time
nyse = mcal.get_calendar('NYSE')
from helpers import *
from tickers import tickers

from scipy.stats import norm as norm
from yahoo_query import *

In [6]:
# '''
# functions list:

#     maturities(dt.datetime()) --> [float(front_wgt), float(back_wgt)]

#     optionslam_scrape(str[ticker]) --> dict[earnings]

#     yahoo_table_parse(str[raw_html_table]) --> DataFrame[ticker]

#     yahoo_earnings(dt.datetime()) --> DataFrame[earnings_on_date]

#     fundamentals(str[ticker]) --> DataFrame[stock_fundamentals]

#     get_fundas(list[ticker_lst]) --> DataFrame[stock_fundamentals]

#     historical_data(str[ticker], int[day_number], int[rolling_window], outsize[str]) --> DataFrame[daily_stock_data]

#     current_volatility(list[ticker_lst], int[roll]) --> DataFrame[stock_volatilities]

#     all_options(str[ticker], bool[greeks]) --> DataFrame[options_chains]

#     earnings_condor(str[ticker], int[max_gap], int[dte_thresh], float[|money_thresh| <= 1]) -- DataFrame[condors], DataFrame[puts], DataFrame[calls]

#     write_excel(str[filename], list[str[sheetnames]], list[dataframes]) --> void()

#     curr_stock_data(str[ticker]) --> DataFrame[stock_info]

#     curr_batch_quotes(list_of_string[tickers]) --> DataFrame[stock_info]

#     past_earnings(str[ticker]) --> DataFrame[earnings_info]

#     earnings_history(str[ticker]) --> [DataFrame[earnings_estimate], DataFrame[past_earnings], DataFrame[earnings_estimate_change]]

#     av_data(str[ticker]) --> DataFrame[ticker_open, ticker_close]

#     av_batch(list_of_str[tickers]) --> DataFrame[tickers_closes]

#     check_mkt_corr(int[corr_rolling_window],int[plot_window]) --> DataFrame[rolling_corr]

#     vvix_check() --> DataFrame[VVIX Data]

#     earnings_longs(list_of_str[ticker], float[bid_ask_spread]) --> DataFrame[option_chains]

#     all_options_v2(str[ticker], int[dte_ub], int[dte_lb], float[moneyness]) --> DataFrame[option_chains]

#     yahoo_options_query(str[ticker], int[dte_ub], int[dte_lb]) --> DataFrame[option_chains]

#     greek_calc(DataFrame[option_chain], str[prem_price_use], str[day_format], float[interest_rate], float[dividend_rate])

#     price_sim(DataFrame[options_df], float[price_change], float[vol_change], int[days_change], str[output = 'All'],
#               str[skew = 'flat'], str[day_format = 'trading'], float[interest_rate = 0.0193], float[dividend_rate = 0],
#               float[prem_price_use = 'Mid'])


#     position_sim(DataFrame[position_df], list_of_int[holdings], int[shares],
#                  float[price_change], float[vol_change], int[dte_change], str[output = 'All'],
#                  str[skew = 'flat'], str[prem_price_use = 'Mid'], str[day_format = 'trading'], 
#                  float[interest_rate = 0.0193], float[dividend_rate = 0])

#     yahoo_fundamentals(list_of_str[tickers]) --> DataFrame[fundamentals]

# '''

stock_list = pd.read_csv('optionablestocks.csv')['OPTION SYMBOL'].tolist()
stock_list = list(set(tickers[3:] + stock_list))

start_time = time.time()

def pull_data(ticker):
    yahoo_data = yahoo_query(ticker,dt.datetime(2018,1,1))
    yahoo_data.full_info_query()
    earnings_info = yahoo_data.earnings_history.join(yahoo_data.cashFlowStatementQuarter).join(yahoo_data.incomeStatementQuarter.drop(['netIncome','maxAge'],
                                                                                                                                      axis = 1),
                                                                                               rsuffix='_income').join(yahoo_data.balanceSheetQuarter,
                                                                                                                       rsuffix = '_balance')
#     earnings_info['earnBeatsBefore'] = 0
#     earnings_info['earnMissBefore'] = 0

#     for idx, row in earnings_info.iterrows():
#         earnings_info.loc[idx,'earnBeatsBefore'] = len(earnings_info[(earnings_info.index <= idx) & (earnings_info.epsDifference > 0)])
#         earnings_info.loc[idx,'earnMissBefore'] = len(earnings_info[(earnings_info.index <= idx) & (earnings_info.epsDifference <= 0)])
#     earnings_info = earnings_info.shift(1)


#     earnings_moves = past_earnings(ticker).sort_index()
#     earnings_moves = earnings_moves[(earnings_moves.index > min(yahoo_data.earnings_history.index) - dt.timedelta(days = 92)) &
#                                     (earnings_moves.index <= max(yahoo_data.earnings_history.index))].sort_index()


#     earnings_df = pd.concat([earnings_info.reset_index(), 
#                              earnings_moves.reset_index()], axis = 1)
    earnings_info.columns = ['quarter' if col == 'index' else col for col in earnings_info.columns.tolist()]
    earnings_info['Underlying'] = ticker

    ### separate df for current key measures
    keyMetrics = yahoo_data.profile.join(yahoo_data.keyStats).join(yahoo_data.finData, rsuffix = '_finData')

    return (earnings_info, keyMetrics)

def download_yahoo_data(ticker_list, retries = 10):

    earnings_lst = []
    keyStats_lst = []

    item_counter = 0
    total_length = len(ticker_list)
    failed_list = []

    for ticker in ticker_list:
        try:
            curr_earnings, curr_keyStats = pull_data(ticker)
            earnings_lst.append(curr_earnings)
            keyStats_lst.append(curr_keyStats)
            print('Accepted: {}'.format(ticker))
        except:
            for i in range(retries):
                try:
                    curr_earnings, curr_keyStats = pull_data(ticker)
                    earnings_lst.append(curr_earnings)
                    keyStats_lst.append(curr_keyStats)
                    print('Accepted: {}'.format(ticker))
                except:
                    continue
            print('Failed: {}'.format(ticker))
            failed_list.append(ticker)

        item_counter += 1
        print('{0:.2f}% Completed'.format(item_counter/total_length*100))
        print('{} total failures'.format(len(failed_list)))

    earnings_df = pd.concat(earnings_lst, axis = 0)
    earnings_df = earnings_df.reset_index()[earnings_df.columns]
    keyStats_df = pd.concat(keyStats_lst, axis = 0)

    return earnings_df, keyStats_df, failed_list

def fin_ratios(earnings_df):

    ratios_df = earnings_df[['Underlying','timestamp','quarter','1Year', '1month', '3month', '6month', 
                             'PostEarningsReturn','industry', 'sector']]

In [9]:
earnings, ks, failed_list = download_yahoo_data(stock_list, retries = 10)
earnings.to_csv('full_data.csv')
ks.to_csv('keyStats_v3.csv')


Accepted: USAT
0.03% Completed
0 total failures
Accepted: RARE
0.05% Completed
0 total failures
Accepted: PHH
0.08% Completed
0 total failures
Accepted: FIT
0.11% Completed
0 total failures
Failed: USMV
0.13% Completed
1 total failures
Failed: FSC
0.16% Completed
2 total failures
Accepted: FIVN
0.19% Completed
2 total failures
Failed: LN
0.22% Completed
3 total failures
Accepted: MPC
0.24% Completed
3 total failures
Failed: MRIN
0.27% Completed
4 total failures
Accepted: HSII
0.30% Completed
4 total failures
Accepted: JILL
0.32% Completed
4 total failures
Accepted: FVE
0.35% Completed
4 total failures
Accepted: AMRN
0.38% Completed
4 total failures
Accepted: TRTN
0.40% Completed
4 total failures
Accepted: LPX
0.43% Completed
4 total failures
Accepted: CARA
0.46% Completed
4 total failures
Accepted: BSRR
0.48% Completed
4 total failures
Failed: SDY
0.51% Completed
5 total failures
Failed: WBAI
0.54% Completed
6 total failures
Failed: INDA
0.57% Completed
7 total failures
Accepted: AEIS


Failed: ELD
4.66% Completed
46 total failures
Accepted: NVCR
4.68% Completed
46 total failures
Accepted: LLY
4.71% Completed
46 total failures
Accepted: TAHO
4.74% Completed
46 total failures
Failed: WFM
4.76% Completed
47 total failures
Accepted: REFR
4.79% Completed
47 total failures
Accepted: BOFI
4.82% Completed
47 total failures
Failed: PSO
4.85% Completed
48 total failures
Failed: ITB
4.87% Completed
49 total failures
Accepted: UPS
4.90% Completed
49 total failures
Accepted: INGN
4.93% Completed
49 total failures
Accepted: WPG
4.95% Completed
49 total failures
Accepted: XLNX
4.98% Completed
49 total failures
Accepted: CRESY
5.01% Completed
49 total failures
Failed: SVVC
5.03% Completed
50 total failures
Accepted: PLAY
5.06% Completed
50 total failures
Accepted: SALT
5.09% Completed
50 total failures
Accepted: CVRR
5.11% Completed
50 total failures
Failed: QQQ
5.14% Completed
51 total failures
Accepted: CIT
5.17% Completed
51 total failures
Accepted: FOE
5.20% Completed
51 total f

Failed: IWD
9.29% Completed
95 total failures
Accepted: KORS
9.31% Completed
95 total failures
Accepted: AVGO
9.34% Completed
95 total failures
Failed: EWN
9.37% Completed
96 total failures
Accepted: EPAY
9.39% Completed
96 total failures
Failed: NOA
9.42% Completed
97 total failures
Failed: DIA
9.45% Completed
98 total failures
Accepted: PBR
9.48% Completed
98 total failures
Accepted: FTI
9.50% Completed
98 total failures
Accepted: FRPT
9.53% Completed
98 total failures
Failed: RWJ
9.56% Completed
99 total failures
Accepted: LXU
9.58% Completed
99 total failures
Accepted: BLK
9.61% Completed
99 total failures
Failed: FXC
9.64% Completed
100 total failures
Accepted: NTRS
9.66% Completed
100 total failures
Failed: GLL
9.69% Completed
101 total failures
Failed: BIV
9.72% Completed
102 total failures
Accepted: CARB
9.74% Completed
102 total failures
Accepted: HT
9.77% Completed
102 total failures
Accepted: BNCL
9.80% Completed
102 total failures
Accepted: HWCC
9.83% Completed
102 total fa

Accepted: LGND
13.76% Completed
146 total failures
Accepted: SLAB
13.78% Completed
146 total failures
Accepted: CNI
13.81% Completed
146 total failures
Accepted: ZAYO
13.84% Completed
146 total failures
Accepted: FLKS
13.86% Completed
146 total failures
Accepted: VMC
13.89% Completed
146 total failures
Accepted: OHRP
13.92% Completed
146 total failures
Accepted: BEN
13.94% Completed
146 total failures
Failed: XLF
13.97% Completed
147 total failures
Accepted: RICK
14.00% Completed
147 total failures
Accepted: NEP
14.02% Completed
147 total failures
Failed: SSRI
14.05% Completed
148 total failures
Accepted: BSX
14.08% Completed
148 total failures
Accepted: HURN
14.10% Completed
148 total failures
Accepted: WMS
14.13% Completed
148 total failures
Accepted: ARE
14.16% Completed
148 total failures
Failed: JNK
14.19% Completed
149 total failures
Accepted: TTGT
14.21% Completed
149 total failures
Accepted: CSTE
14.24% Completed
149 total failures
Accepted: HRB
14.27% Completed
149 total failu

Failed: CURE
18.20% Completed
192 total failures
Accepted: MXWL
18.22% Completed
192 total failures
Accepted: DISCA
18.25% Completed
192 total failures
Accepted: CRMD
18.28% Completed
192 total failures
Failed: AVYA
18.30% Completed
193 total failures
Accepted: IMMR
18.33% Completed
193 total failures
Failed: WEAT
18.36% Completed
194 total failures
Failed: VPU
18.38% Completed
195 total failures
Accepted: HBM
18.41% Completed
195 total failures
Accepted: CAMP
18.44% Completed
195 total failures
Accepted: VMW
18.47% Completed
195 total failures
Accepted: HP
18.49% Completed
195 total failures
Accepted: LGIH
18.52% Completed
195 total failures
Accepted: EDU
18.55% Completed
195 total failures
Failed: UGL
18.57% Completed
196 total failures
Failed: ROP
18.60% Completed
197 total failures
Failed: RT
18.63% Completed
198 total failures
Failed: SMSI
18.65% Completed
199 total failures
Accepted: DS
18.68% Completed
199 total failures
Accepted: QRVO
18.71% Completed
199 total failures
Accepte

...

Failed: BWX
80.35% Completed
889 total failures
Accepted: PNK
80.38% Completed
889 total failures
Accepted: INTC
80.40% Completed
889 total failures
Accepted: MA
80.43% Completed
889 total failures
Accepted: RDWR
80.46% Completed
889 total failures
Accepted: KMX
80.48% Completed
889 total failures
Accepted: SKT
80.51% Completed
889 total failures
Failed: VXUS
80.54% Completed
890 total failures
Accepted: SBSI
80.57% Completed
890 total failures
Accepted: MEET
80.59% Completed
890 total failures
Failed: PDFS
80.62% Completed
891 total failures
Accepted: BMS
80.65% Completed
891 total failures
Accepted: EQIX
80.67% Completed
891 total failures
Accepted: APD
80.70% Completed
891 total failures
Accepted: CCE
80.73% Completed
891 total failures
Failed: BTI
80.75% Completed
892 total failures
Failed: TYO
80.78% Completed
893 total failures
Accepted: GDDY
80.81% Completed
893 total failures
Accepted: ENVA
80.83% Completed
893 total failures
Failed: JASO
80.86% Completed
894 total failures
Acc

Failed: GIMO
84.79% Completed
940 total failures
Failed: CSJ
84.82% Completed
941 total failures
Accepted: ETP
84.85% Completed
941 total failures
Failed: IGE
84.87% Completed
942 total failures
Accepted: PYPL
84.90% Completed
942 total failures
Failed: EVEP
84.93% Completed
943 total failures
Accepted: TCO
84.95% Completed
943 total failures
Accepted: MEOH
84.98% Completed
943 total failures
Failed: SOCL
85.01% Completed
944 total failures
Accepted: FLWS
85.03% Completed
944 total failures
Accepted: NCLH
85.06% Completed
944 total failures
Accepted: PEGA
85.09% Completed
944 total failures
Failed: ICF
85.11% Completed
945 total failures
Accepted: SYMC
85.14% Completed
945 total failures
Accepted: LOW
85.17% Completed
945 total failures
Accepted: QADA
85.20% Completed
945 total failures
Accepted: CY
85.22% Completed
945 total failures
Accepted: BFR
85.25% Completed
945 total failures
Failed: IWN
85.28% Completed
946 total failures
Accepted: GKOS
85.30% Completed
946 total failures
Fail

Accepted: HPQ
89.23% Completed
985 total failures
Accepted: MX
89.26% Completed
985 total failures
Accepted: HII
89.29% Completed
985 total failures
Failed: ZSL
89.31% Completed
986 total failures
Failed: CPL
89.34% Completed
987 total failures
Accepted: BMRN
89.37% Completed
987 total failures
Failed: NVDQ
89.39% Completed
988 total failures
Accepted: AVA
89.42% Completed
988 total failures
Accepted: HUN
89.45% Completed
988 total failures
Accepted: RUN
89.48% Completed
988 total failures
Accepted: SXE
89.50% Completed
988 total failures
Accepted: NWSA
89.53% Completed
988 total failures
Failed: RWO
89.56% Completed
989 total failures
Accepted: BURL
89.58% Completed
989 total failures
Accepted: MAA
89.61% Completed
989 total failures
Accepted: UNT
89.64% Completed
989 total failures
Accepted: TNP
89.66% Completed
989 total failures
Accepted: TLP
89.69% Completed
989 total failures
Accepted: BHE
89.72% Completed
989 total failures
Failed: IHF
89.74% Completed
990 total failures
Accepte

Accepted: BAM
93.59% Completed
1024 total failures
Accepted: MDU
93.62% Completed
1024 total failures
Failed: RWX
93.65% Completed
1025 total failures
Failed: CPIX
93.67% Completed
1026 total failures
Failed: XHB
93.70% Completed
1027 total failures
Accepted: SPPI
93.73% Completed
1027 total failures
Failed: CZZ
93.76% Completed
1028 total failures
Accepted: NS
93.78% Completed
1028 total failures
Failed: BLV
93.81% Completed
1029 total failures
Accepted: HA
93.84% Completed
1029 total failures
Accepted: FELP
93.86% Completed
1029 total failures
Failed: ATW
93.89% Completed
1030 total failures
Failed: DBEU
93.92% Completed
1031 total failures
Failed: BWLD
93.94% Completed
1032 total failures
Accepted: FCEL
93.97% Completed
1032 total failures
Accepted: MTGE
94.00% Completed
1032 total failures
Accepted: NUVA
94.02% Completed
1032 total failures
Accepted: AOBC
94.05% Completed
1032 total failures
Accepted: AMP
94.08% Completed
1032 total failures
Accepted: CBB
94.10% Completed
1032 tota

Accepted: ERI
97.95% Completed
1076 total failures
Accepted: IVAC
97.98% Completed
1076 total failures
Accepted: QUIK
98.01% Completed
1076 total failures
Accepted: VNO
98.03% Completed
1076 total failures
Accepted: QGEN
98.06% Completed
1076 total failures
Accepted: GMS
98.09% Completed
1076 total failures
Accepted: WAGE
98.12% Completed
1076 total failures
Accepted: INFI
98.14% Completed
1076 total failures
Accepted: RFP
98.17% Completed
1076 total failures
Accepted: SCG
98.20% Completed
1076 total failures
Accepted: LNDC
98.22% Completed
1076 total failures
Accepted: CZR
98.25% Completed
1076 total failures
Failed: DDM
98.28% Completed
1077 total failures
Accepted: ARNC
98.30% Completed
1077 total failures
Accepted: AMSC
98.33% Completed
1077 total failures
Accepted: SDRL
98.36% Completed
1077 total failures
Failed: RDNT
98.38% Completed
1078 total failures
Accepted: TWO
98.41% Completed
1078 total failures
Accepted: PRTA
98.44% Completed
1078 total failures
Failed: PTM
98.47% Compl

In [ ]:
ks = pd.read_csv('keyStats.csv',index_col = 0).drop_duplicates()
earnings = pd.read_csv('earningsHist_unshifted.csv', index_col = 0).drop_duplicates()

In [ ]:
ks = ks.drop(ks.T.isnull().sum(axis = 1).sort_values(ascending = False).head(18).index.tolist(),
             axis = 1)

score_cols = ['52WeekChange', 'earningsGrowth',
              'earningsQuarterlyGrowth', 'forwardPE',
              'grossMargins', 'grossProfits',
              'pegRatio', 'profitMargins', 
              'returnOnAssets',
              'returnOnEquity', 'revenueGrowth', 'sector']




earnings.quarter = pd.to_datetime(earnings.quarter)
earnings = earnings.groupby(['Underlying','quarter']).sum().reset_index()
ticker = 'A'
latest_earnings = []
for ticker in earnings.Underlying.drop_duplicates().tolist():
    latest_earnings.append(earnings[earnings['Underlying'] == ticker][earnings.quarter == max(earnings[earnings['Underlying'] == ticker].quarter)])
latest_earnings = pd.concat(latest_earnings, axis = 0)
latest_earnings.index = latest_earnings.Underlying

earn_cols = ['quarter', '1Year', '1month', '3month', '6month','totalLiab',
             'totalStockholderEquity','netIncome',
             'operatingIncome',
             'totalRevenue','dividendsPaid',
             'investments',
             'longTermInvestments',
             'shortTermInvestments']

latest_earnings = latest_earnings[earn_cols]


ks = ks[score_cols]

names = ks.join(latest_earnings).drop_duplicates()

names['dividendsPaid'] = -names['dividendsPaid']

names['payoutRatio'] = names.dividendsPaid/names.netIncome
names['debtEquityRatio'] = names.totalLiab/names.totalStockholderEquity
names['netMargin'] = names.netIncome/names.totalRevenue
names['roic'] = names.operatingIncome/(names.longTermInvestments + names.shortTermInvestments)

names = names[['debtEquityRatio',
       '52WeekChange','earningsGrowth',
       'earningsQuarterlyGrowth', 'forwardPE',
       'grossMargins', 'grossProfits',
       'pegRatio', 'profitMargins', 
       'returnOnAssets',
       'returnOnEquity', 'revenueGrowth', 'sector']]

names = names[names['52WeekChange'] != 'Infinity']
names['52WeekChange'] = pd.to_numeric(names['52WeekChange'])

removes = pd.DataFrame(names.isnull().sum(axis = 1).sort_values(ascending = False))
removes = removes[removes[0] <= 3]

names = names[names.index.isin(removes.index.tolist())].fillna(0)

names_mean = names.groupby('sector').mean()
names_mean = names[['sector']].join(names_mean, on = 'sector')

names_high = names.groupby('sector').max()
names_high = names[['sector']].join(names_high, on = 'sector')

names_low = names.groupby('sector').min()
names_low = names[['sector']].join(names_low, on = 'sector')

names_std = names.groupby('sector').std()
names_std = names[['sector']].join(names_std, on = 'sector')

del names['sector'], names_mean['sector'], names_high['sector'], names_low['sector'], names_std['sector']

score = (names - names_mean)/names_std#(names_high - names_low)

del score['forwardPE']

score['score'] = ((score['52WeekChange'])/3 -
                   (score['debtEquityRatio'] + score['pegRatio'])/3 +
                   (score['earningsGrowth'] + score['revenueGrowth'])/3)
score.join(ks[['sector']]).sort_values('score',ascending = False).to_csv('leaps_scores.csv')